# Procesamiento de Lenguaje Natural - **Deploy de un modelo**

En esta notebook vamos a jugar con la creación de un modelo basados en el procesamiento que hicimos en notebooks pasadas y vamos a hacer un mini deploy de dicho modelo.

*Nota*. Para los efectos de esta notebook vamos a usar una estrategia simple de división del dataset en training-test. Se podrían optar por otras opciones u otros esquemas de división. Asimismo, si bien se muestra una posibilidad para la elección del "mejor" modelo mediante una optimización de parámetros, no será utilizada para el modelo a guardar.

### Creación del modelo

Primero, vamos a traernos los datos con los que vamos a estar trabajando. Vamos a seguir utilizando el dataset simple de detección de hate speech, del que nos vamos a quedar con un atributo de tipo texto y la clase numérica (0: No es hate speech, 1: hate speech, 2: offensive speech).

In [ ]:
# Cargamos los datos necesarios
import pandas as pd

url = "https://raw.githubusercontent.com/t-davidson/hate-speech-and-offensive-language/master/data/labeled_data.csv"
df = pd.read_csv(url, usecols=['class', 'tweet']) # de todas las columnas que tiene el dataset, nos vamos a quedar solo con el texto y la clase

print(df[:1000]) # limitamos la cantidad de instancias para que no tarde ni el pre-processing ni el training

     class                                              tweet
0        2  !!! RT @mayasolovely: As a woman you shouldn't...
1        1  !!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2        1  !!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3        1  !!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4        1  !!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...
..     ...                                                ...
995      1  &#128514;&#128514;&#128514;&#128514; RT @SMASH...
996      1  &#128514;&#128514;&#128514;&#128514; bitch if ...
997      1  &#128514;&#128514;&#128514;&#128514; these fol...
998      1  &#128514;&#128514;&#128514;&#128514;&#128514; ...
999      1  &#128514;&#128514;&#128514;&#128514;&#128514;&...

[1000 rows x 2 columns]


In [ ]:
from sklearn.model_selection import train_test_split

train_set, test_set = train_test_split(df, test_size = 0.80,random_state=42) # limitamos el tamaño del training para que no tarde

# recordemos que para entrenar tenemos separar la clase
X_train = train_set.drop("class", axis=1)  
y_train = train_set["class"].copy()

X_test = test_set.drop("class",axis=1) # nos dejamos también preparado el test set
y_test = test_set["class"].copy() # nos dejamos también preparado el test set

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
import nltk
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
count_normal = CountVectorizer(stop_words=nltk.corpus.stopwords.words('english'))

preprocessor = ColumnTransformer(
    transformers=[
        ('count', count_normal, "tweet")]) # importante definir las columnas sobre las cuales se aplica

rf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', SVC(probability=True))])

Finalmente, vamos a entrenar el modelo. De acuerdo al modelo que elijamos, puede tardar.

In [ ]:
rf.fit(X_train,y_train)      

Pipeline(memory=None,
         steps=[('preprocessor',
                 ColumnTransformer(n_jobs=None, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('count',
                                                  CountVectorizer(analyzer='word',
                                                                  binary=False,
                                                                  decode_error='strict',
                                                                  dtype=<class 'numpy.int64'>,
                                                                  encoding='utf-8',
                                                                  input='content',
                                                                  lowercase=True,
                                                                  max_df=1.0,
                            

In [ ]:
rf.predict(X_test)

array([1, 1, 1, ..., 1, 1, 1])

### Model selection

El pipeline que definimos también puede ser utilizado en el proceso de selección de modelos. En el siguiente fragmento de código se cicla por diferentes modelos de clasificación provistos por sklearn, para aplicar las transformaciones y luego entrenarlos.

Nota. Hay más clasificadores disponibles para probar.

Nota 2. Puede tardar!!

In [ ]:
from sklearn.metrics import accuracy_score

from sklearn.neighbors import KNeighborsClassifier 
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

classifiers = [
    KNeighborsClassifier(3),
    DecisionTreeClassifier(),
    RandomForestClassifier(),
    SVC()
    ]

for classifier in classifiers:
    pipe = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', classifier)])
    pipe.fit(X_train, y_train)   
    print(classifier)
    print("model score: %.3f" % pipe.score(X_test, y_test)) # qué retorna depende del modelo que se usa. En clasificación retorna accuracy promedio

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=3, p=2,
                     weights='uniform')
model score: 0.822
DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')
model score: 0.885
RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split

Finalmente, el pipeline que definimos también puede ser utilizado en un grid search para encontrar la mejor combinación de hiper-parámetros.

Para hacer esto, lo primero que hay que hacer es crear una grilla de parámetros para el modelo elegido. Algo importante a notar es que a los nombres de los parámetros hay que agregarles el nombre que le dimos al parámetro que representaba al algoritmo (en este caso de clasificación, al que llamamos ``classifier``).

Luego, creamos el objeto de grid search el cual incluye el pipeline original. Cuando llamemos al método ``fit``, antes de realizar la búsqueda del grid search se aplicarán las transformaciones.

Nota. En este ejemplo se están considerando dos parámetros para el ``RandomForestClassifier``. De acuerdo al clasificador, los parámetros que se podrán optimizar.

Nota 2. Hay múltiples métricas de [scoring](https://scikit-learn.org/stable/modules/model_evaluation.html#scoring) que pueden ser consideradas. Ver también la documentación referida a [model evaluation](https://scikit-learn.org/stable/modules/model_evaluation.html).

Nota 3. Puede tardar!!

In [ ]:
from sklearn.model_selection import GridSearchCV

rfcv = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', RandomForestClassifier())])

param_grid = { 
    'classifier__n_estimators': [1, 3, 10],
    'classifier__max_features': ['auto', 'sqrt'],
}

CV = GridSearchCV(rfcv, param_grid, cv=5,
                           scoring='f1_weighted') 
                  
CV.fit(X_train, y_train)  
print(CV.best_params_)    
print(CV.best_score_)

{'classifier__max_features': 'sqrt', 'classifier__n_estimators': 3}
0.8421755733496846


#### Y cómo sabemos cuál es el "mejor" modelo?

Ya hicimos el test de nuestros modelos varias veces, tenemos los valores de diversas métricas para cada ejecución de cada modelo, cómo sabemos ahora cuál es el mejor? Comparar los valores individuales de las métricas puede no ser suficiente. 

Supongamos que hicimos 10 ejecuciones para cada modelo, promediamos los resultados, nos quedamos con los promedios. Podemos asegurar que si la métrica del modelo A nos da mayor que la del modelo B (suponiendo que la métrica a mayor valor, mejor resultado) el modelo A es mejor que el modelo B? Por ejemplo, si el modelo A tiene un resultado de 0.88 y el modelo B un resultado de 0.86, podemos afirmar que el modelo A es mejor que el B. La respuesta es: **no necesariamente**.

* Las métricas de performance pueden no ser el único criterio que tenemos para analizar el modelo. También pueden entrar en juego cuestiones como complejidad computacional, recursos consumidos, evaluaciones subjetivas de los usuarios, métricas de diversidad, etc.

* No todas las diferencias observables entre los modelos son significativas. Que se observe una diferencia (como en el caso de 0.88 y 0.86) no quiere decir que la diferencia no se haya debido a la casualidad o al azar.


Entonces, qué se puede hacer en estas situaciones? Aplicar tests estadísticos.



In [ ]:
import scipy.stats

In [ ]:
metrics_model_A = [0.516129, 0.444444, 0.631579, 0.516129, 0.545455, 0.344828, 0.5, 0.533333, 0.594595, 0.428571] # asumamos que estos son los resultados de la métrica X de haber ejecutado 10 veces el modelo A
metrics_model_B = [0.516129, 0.645161, 0.571429, 0.4, 0.533333, 0.4375, 0.428571, 0.387097, 0.529412, 0.545455] # asumamos que estos son los resultados de la métrica X de haber ejecutado 10 veces el modelo B

Lo primero que hay que testear es normalidad, para saber qué tipo de test utilizar. En este test de normalidad hay que mirar el valor del ``p-value``. Si el ``p-value`` es menor que el valor de confianza ``alpha`` que se define (usualmente en ``0.01`` o ``0.05``), se puede rechazar la hipótesis nula de que las distribuciones son normales. Por el contrario, si el ``p-value`` es mayor que el ``alpha``, se debe asumir que la distribución es normal.

In [ ]:
print(scipy.stats.normaltest(metrics_model_A))
print(scipy.stats.normaltest(metrics_model_B))

NormaltestResult(statistic=0.821362046254129, pvalue=0.6631984428307685)
NormaltestResult(statistic=0.30729389511497246, pvalue=0.8575747364264382)


/usr/local/lib/python3.6/dist-packages/scipy/stats/stats.py:1535: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=10
  "anyway, n=%i" % int(n))


Una vez que sabemos qué tipo de distribuciones tenemos, podemos aplicar el test.

En este caso, asumimos que las ejecuciones para el modelo A y el modelo B fueron realizados para los mismas particiones de los datos, con lo que vamos a utilizar tests ``paired``. Es decir, por ejemplo, la ejecución 1 de ambos modelos se realizó con la partición de datos X1.

En estos tests, hay que mirar de nuevo el ``p-value``. Si ``p-value < alpha``, podemos rechazar la hipótesis nula de que las diferencias entre las distribuciones no son significativas. Por el contrario, si ``p-value >= alpha``, no podemos rechazarla y debemos asumir que no hay una diferencia estadística entre las distribuciones.

In [ ]:
print(scipy.stats.ttest_rel(metrics_model_A,metrics_model_B)) # en el caso en el que las dos distribuciones sea normal

print(scipy.stats.wilcoxon(metrics_model_A,metrics_model_B)) # en el caso en el que al menos una de las distribuciones no sea normal.

Ttest_relResult(statistic=0.17429440352789408, pvalue=0.865491728742791)
WilcoxonResult(statistic=21.0, pvalue=0.8589549227374824)


/usr/local/lib/python3.6/dist-packages/scipy/stats/morestats.py:2879: UserWarning: Sample size too small for normal approximation.
  warnings.warn("Sample size too small for normal approximation.")


Vamos a probar con otras distribuciones

In [ ]:
metrics_model_A = [0.516129, 0.444444, 0.631579, 0.516129, 0.545455, 0.344828, 0.5, 0.533333, 0.594595, 0.428571] # asumamos que estos son los resultados de la métrica X de haber ejecutado 10 veces el modelo A
metrics_model_B = [0.693878, 0.666667, 0.666667, 0.693878, 0.666667, 0.571429, 0.666667, 0.555556, 0.5, 0.693878]

print(scipy.stats.normaltest(metrics_model_A))
print(scipy.stats.normaltest(metrics_model_B))

NormaltestResult(statistic=0.821362046254129, pvalue=0.6631984428307685)
NormaltestResult(statistic=3.0192637321155904, pvalue=0.2209913173914211)


/usr/local/lib/python3.6/dist-packages/scipy/stats/stats.py:1535: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=10
  "anyway, n=%i" % int(n))


In [ ]:
print(scipy.stats.ttest_rel(metrics_model_A,metrics_model_B))

Ttest_relResult(statistic=-3.7233729994558984, pvalue=0.004745815527517936)


Nota. Accuracy puede ser la primera métrica que se nos ocurre para evaluar la performance de un clasificador, pero puede que no sea la más adecuada en todos los contextos. Hay otras métricas más robustas que pueden darnos una mejor idea de cómo funciona nuestro modelo.

### Salvar/Exportar el modelo

En este caso el modelo lo necesitamos solo acá, pero qué pasaría si nosotros quisieramos llevar este modelo a otro ambiente o simplemente reemplazar otro modelo que teníamos por este. Tenemos que repetir todos los pasos de definición del pipeline y reentrenar? No, no es necesario.

Lo que podemos hacer es persistir el modelo, es decir, guardarlo en un archivo que luego podremos levantar en donde nosotros quisiéramos utilizarlo.

La primera alternativa es usar ```joblib```.

In [ ]:
import joblib

joblib.dump(rf, "hate_speech_detection_model.pkl") 

['hate_speech_detection_model.pkl']

Luego, para cargarlo:

In [ ]:
loaded_model = joblib.load("hate_speech_detection_model.pkl")
y_pred = loaded_model.predict(X_test)
print(y_pred)

[1 1 1 ... 1 1 1]


Otra alternativa es usar ```Pickle```.

In [ ]:
import pickle

# save the model to disk
filename = 'finalized_model.sav'
pickle.dump(rf, open(filename, 'wb'))

Luego, para cargarlo:

In [ ]:
# load the model from disk
loaded_model = pickle.load(open(filename, 'rb'))
y_pred = loaded_model.predict(X_test)
print(y_pred)

[1 1 1 ... 1 1 1]


### Deploy del modelo

Para hacer el deploy del modelo, vamos a crear una aplicación de hate speech detection para deployarla como un servicio REST básandonos en el modelo que creamos en los bloques anteriores.

Vamos a usar:

* [Flask](https://github.com/pallets/flask): uno de los micro web frameworks más populares.
* [flask_ngrok](https://pypi.org/project/flask-ngrok/): herramienta que nos permite hacer demos de nuestras apps Flask. Nos permite servir nuestra applicación desde una simple notebook.

Instalamos las dependencias que vamos a necesitar

In [ ]:
!pip install flask
!pip install flask-ngrok

Lo primero que vamos a hacer es dejar accessible nuestro modelo entrenado a nuestra aplicación. 

En este caso, si venimos ejecutando toda la notebook vamos a tener disponible nuestro ``hate_speech_detection_model.pkl``.

Nota. Cada vez que reiniciemos el runtime, deberíamos generar o levantar nuestro modelo de algún lado.

In [ ]:
from flask_ngrok import run_with_ngrok
from flask import Flask,request,jsonify
import pandas as pd 
import pickle
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.externals import joblib

app = Flask(__name__)

run_with_ngrok(app)

@app.route('/')
def home():
  return "<h1>Hate Speech Detector!</h1>"

model = joblib.load("hate_speech_detection_model.pkl") # vamos a levantar nuestro modelo

@app.route('/predict',methods=['GET','POST']) # los tipos de métodos que soportamos
def predict():
  
  df_n = pd.DataFrame({"tweet":[request.args['text']]}) # tomamos el texto que nos pasaron para hacer la predicción

  prediction = model.predict(df_n) # utilizamos el modelo para predecir
  pred_proba = model.predict_proba(df_n) # obtenemos la probabilidad de la predicción --> No está disponible para todos los clasificadores!

  if prediction == 0:
    pred_text = 'Hate'
  elif prediction == 1:
    pred_text = 'Offensive'
  else:
    pred_text = 'Neither'   

  print(request.args['text']) # en la consola de acá imprimimos el texto
  print(pred_proba.dtype) # en la consola de acá imprimimos la probabilidad

  return "<h2>El texto \""+request.args['text']+"\" fue clasificado como: "+pred_text+" con una probabilidad de: "+str(pred_proba[0][prediction])+"</h2>"

app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)
 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://1bf944b8874a.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [15/Aug/2020 17:34:01] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [15/Aug/2020 17:34:02] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [15/Aug/2020 17:34:02] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [15/Aug/2020 17:34:02] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [15/Aug/2020 17:34:03] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [15/Aug/2020 17:34:04] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [15/Aug/2020 17:34:05] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [15/Aug/2020 17:34:05] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [15/Aug/2020 17:34:05] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [15/Aug/2020 17:34:06] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [15/Aug/2020 17:34:08] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [15/Aug/2020 17:34:08] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [15/Aug/2020 17:34:11] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [15/Aug/2020 17:34:12] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [15/Aug/2020 17:34:13] "GET /predict?text=I%20hate%20everyone HTTP/1.

I hate everyone
float64


...

127.0.0.1 - - [15/Aug/2020 17:34:39] "GET /predict?text=i%27m_trying_this_text HTTP/1.1" 200 -
127.0.0.1 - - [15/Aug/2020 17:34:39] "GET /Hate HTTP/1.1" 404 -


i'm_trying_this_text
float64


127.0.0.1 - - [15/Aug/2020 17:34:39] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [15/Aug/2020 17:34:41] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [15/Aug/2020 17:34:44] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [15/Aug/2020 17:34:44] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [15/Aug/2020 17:34:54] "GET /predict?text=I%20hate%20Twitter HTTP/1.1" 200 -


I hate Twitter
float64


127.0.0.1 - - [15/Aug/2020 17:34:54] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [15/Aug/2020 17:34:54] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [15/Aug/2020 17:34:55] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [15/Aug/2020 17:35:03] "GET / HTTP/1.1" 200 -


Y listo! Ahora ya tenemos nuestro detector de hate speech disponible! Si le pasamos un argumento ```text``` con un string nos va a retornar si es o no hate speech! Para pasarle el argumento ``URL_NGROK/predict?text=TEXTO``. Por ejemplo, ``http://b36fd66da979.ngrok.io/predict?text="I hate everyone!!"``

El formato de salida que le dimos no es muy amigable con el usuario si lo que queremos es dejarlo disponible y que otros lo usen. Para eso, vamos a modificar la salida para que nos retorne un json.

In [ ]:
# es el mismo código que antes, solo cambia el return

from flask_ngrok import run_with_ngrok
from flask import Flask,request,jsonify
import pandas as pd 
from sklearn.externals import joblib
import json

app = Flask(__name__)

run_with_ngrok(app)

@app.route('/')
def home():
  return "<h1>Hate Speech Detector!</h1>"

model = joblib.load("hate_speech_detection_model.pkl") 

@app.route('/predict',methods=['GET','POST'])
def predict():
  
  df_n = pd.DataFrame({"tweet":[request.args['text']]})

  prediction = model.predict(df_n)
  pred_proba = model.predict_proba(df_n)

  if prediction == 0:
    pred_text = 'Hate'
  elif prediction == 1:
    pred_text = 'Offensive'
  else:
    pred_text = 'Neither'  

  output = {'text': request.args['text'], 'prediction': pred_text, 'confidence': str(pred_proba[0][prediction])}

  return json.dumps(output)

app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://fdf3de0310f0.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [15/Aug/2020 17:35:57] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [15/Aug/2020 17:35:58] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [15/Aug/2020 17:35:58] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [15/Aug/2020 17:35:58] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [15/Aug/2020 17:35:59] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [15/Aug/2020 17:35:59] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [15/Aug/2020 17:35:59] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [15/Aug/2020 17:36:00] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [15/Aug/2020 17:36:00] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [15/Aug/2020 17:36:00] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [15/Aug/2020 17:36:00] "GET /predict?text=I%20hate%20everyone HTTP/1.1" 200 -
127.0.0.1 - - [15/Aug/2020 17:36:01] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [15/Aug/2020 17:36:09] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [15/Aug/2020 17:36:10] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [15/Aug/2020 17:36:25] "GET /predict?

Si queremos probar de consumirlo como un servicio "normal", podemos ejecutar el siguiente código (en otra notebook, dado que acá estamos ejecutando el server).

In [ ]:
import requests

text = "I think you are not pretty"
base = "COMPLETAR_URL_NGROK"
url = base+'/predict?text='+text

response = requests.post(url)
print(response.content)

Otra posibilidad sería esto mismo deployarlo en algún proveedor Cloud o incluso hacerlo accesible como una imagen de Docker.

